# Interação
---

>_“A graphic is not ‘drawn’ once and for all; it is ‘constructed’ and reconstructed until it reveals all the relationships constituted by the interplay of the data. The best graphic operations are those carried out by the decision-maker themself.”_ &mdash; [Jacques Bertin](https://books.google.com/books?id=csqX_xnm4tcC)
>>_“Um gráfico não é ‘desenhado’ de uma vez por todas; ele é ‘construído’ e reconstruído até revelar todas as relações constituídas pela interação dos dados. As melhores operações gráficas são aquelas realizadas pelo próprio tomador de decisão.”_ 

Visualizações oferecem um caminho poderoso para a compreensão de dados. Uma única imagem, no entanto, tipicamente responde, na melhor das hipóteses, um pequeno conjunto de perguntas. Por meio da _interação_, podemos transformar imagens estáticas em ferramentas de exploração: destacando pontos de interesse, mudando a perspectiva para revelar padrões mais detalhados e vinculando múltiplas visões para analisar relações multidimensionais.  


No cerne da interação está a noção de _seleção_: um meio de indicar ao computador quais elementos ou regiões nos interessam. Por exemplo, podemos passar o mouse sobre um ponto, clicar em múltiplas marcas ou desenhar uma caixa delimitadora ao redor de uma região para destacar subconjuntos dos dados para análise mais aprofundada.  

Além das codificações visuais e transformações de dados, o Altair fornece uma abstração de _seleção_ para a criação de interações. Essas seleções abrangem três aspectos:  

1. Manipulação dos inputs para selecionar pontos ou regiões de interesse, como passar o mouse, clicar, arrastar, rolar e tocar (telas sensíveis).  
2. Generalização da entrada para formar uma regra de seleção (ou [_predicado_](https://en.wikipedia.org/wiki/Predicate_%28mathematical_logic%29)) que determina se um determinado registro de dados está ou não dentro da seleção.  
3. Uso do predicado de seleção para configurar dinamicamente uma visualização, acionando _codificações condicionais_, _transformações de filtro_ ou _domínios de escala_.  

Este capítulo apresenta seleções interativas e explora como usá-las para desenvolver diversas técnicas de interação, como consultas dinâmicas, panorâmica &amp; zoom, detalhes sob demanda e brushing &amp; linking.

_Esse capítulo faz parte do [currículo de visualização de dados](https://github.com/uwdata/visualization-curriculum)._

In [ ]:
import altair as alt
import pandas as pd

## Datasets
---

Nós vamos visuzalisar uma variedade de datasets da coleção [vega-datasets](https://github.com/vega/vega-datasets):

- O dataset `cars` com modelos entre 1970 e começos dos anos 1980.
- O dataset `movies`, usado previamente no capítulo [Data Transformation](https://github.com/uwdata/visualization-curriculum/blob/master/altair_data_transformation.ipynb),
- O dataset contendo 10 anos de cotações de ações [S&amp;P 500](https://en.wikipedia.org/wiki/S%26P_500_Index) (`sp500`),
- O dataset das cotações de empresas de tecnologia `stocks` e
- O dataset `flights`, incluindo hora de partida, distância e atraso de chegada.

In [ ]:
cars = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/cars.json'
movies = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'
sp500 = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/sp500.csv'
stocks = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/stocks.csv'
flights = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/flights-5k.json'

## Introduzindo seleções
---

Vamos começar com uma seleção básica: simplesmente clicar em um ponto para destacá-lo. Usando o _dataset_ `cars`, criaremos um gráfico de dispersão (_scatter plot_) relacionando potência (_horsepower_) e milhas por galão (_miles per gallon_), com uma codificação de cor baseada no número de cilindros do motor do carro.  

Além disso, criaremos uma instância de seleção chamando `alt.selection_single()`, indicando que queremos que a seleção seja definida sobre um _único valor_. Por padrão, a seleção usa um clique do mouse para determinar o valor selecionado. Para registrar uma seleção em um gráfico, devemos adicioná-la usando o método `.add_selection()`.  

Uma vez definida a seleção, podemos utilizá-la como parâmetro para _codificações condicionais_, que aplicam uma codificação diferente dependendo se um registro de dados está dentro ou fora da seleção. Por exemplo, considere o seguinte código:  

~~~ python
color=alt.condition(selection, 'Cylinders:O', alt.value('grey'))
~~~  

Essa definição de codificação especifica que os pontos de dados contidos na `selection` devem ser coloridos de acordo com o campo `Cylinders`, enquanto os pontos não selecionados devem usar a cor padrão `grey`. Uma seleção vazia inclui _todos_ os pontos de dados, de modo que, inicialmente, todos os pontos serão coloridos.  

_Tente clicar em diferentes pontos no gráfico abaixo. O que acontece? (Clique no fundo resetar a seleção.)_

In [ ]:
selection = alt.selection_single();
  
alt.Chart(cars).mark_circle().add_selection(
    selection
).encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(selection, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
)

É claro que destacar pontos de dados um a um não é particularmente promissor! Entretanto, como veremos, seleções de um único valor são um útil bloco de construção de interações mais poderosas. Of course, highlighting individual data points one-at-a-time is not particularly exciting! As we'll see, however, single value selections provide a useful building block for more powerful interactions. Além disso, seleções de um único valor só podem pertencer a um desses três tipos fornecidos pelo Altair:

- `selection_single` - seleciona um único valor discreto, por configuração inicial através de cliques. 
- `selection_multi` - seleciona múltiplos valores discretos. O primeiro é selcionado por um clique de mouse e os seguintes, values. The first value is selected on mouse click and additional values toggled using shift-click.
- `selection_interval` - seleciona um intervalo contínuo de valores e é iniciado pelo arrastar do mouse. 

Vamos comparar cada um desses tipos de seleção lado a lado. Para manter seu código organizado, vamos primeiro definir uma função (`plot`) que gera uma especificação para gráfico de dispersão (_scatter plot_) exatamente como a que vimos acima. Podemos passar uma seleção para a função `plot` Para aplicá-la ao gráfico:

In [ ]:
def plot(selection):
    return alt.Chart(cars).mark_circle().add_selection(
        selection
    ).encode(
        x='Horsepower:Q',
        y='Miles_per_Gallon:Q',
        color=alt.condition(selection, 'Cylinders:O', alt.value('grey')),
        opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
    ).properties(
        width=240,
        height=180
    )

Vamos usar nossa função `plot` para criar três variantes de gráficos, uma para cada tipo de seleção.

O primeiro (`single`) gráfico replica nosso exemplo anterior. O segundo (`multi`) gráfico aceita interações shift-click para alternar inclusão de múltiplos pontos dentro da seleção. O terceiro (`interval`) gráfico gera uma seleção de região (ou _brush_) pelo arrastar do mouse. Uma vez criada, você pode arrastar o pincel por aí para selecionar diferentes pontosnce created, ou rodar quando o cursor está dentro da região para mudar a escala (zoom) do pincel.

_Tente interagir com cada um dos gráficos abaixo!_

In [ ]:
alt.hconcat(
  plot(alt.selection_single()).properties(title='Single (Click)'),
  plot(alt.selection_multi()).properties(title='Multi (Shift-Click)'),
  plot(alt.selection_interval()).properties(title='Interval (Drag)')
)

Os exemplos acima utilizam interações padrão (clique, shift-clique, arrastar) para cada tipo de seleção. Podemos customizar as interações mais a fundo ao fonecer especificações de input usando a [sintaxe de seleção de eventos do Vega](https://vega.github.io/vega/docs/event-streams/). Por exemplo, podemos modificar nossos gráficos `single` e `multi` para que tenham por gatilho eventos de `passagem de mouse` _(`mouseover`)_ ao invés dos de `clique`.

_Mantenha pressionada a tecla Shift no segundo gráfico para "pintar" com os dados!_

In [ ]:
alt.hconcat(
  plot(alt.selection_single(on='mouseover')).properties(title='Single (Mouseover)'),
  plot(alt.selection_multi(on='mouseover')).properties(title='Multi (Shift-Mouseover)')
)

Agora que cobrimos as bases das seleções em Altair, vamos por um tour através das várias técnicas de interação que encerram!

## Consultas dinâmicas 
---

Introduzindo seleções

_Consultas dinâmicas_ permitem a exploração rápida e reversível de dados para isolar padrões de interesse. Conforme definido por [Ahlberg, Williamson e Shneiderman](https://www.cs.umd.edu/~ben/papers/Ahlberg1992Dynamic.pdf), uma consulta dinâmica:

- representa uma consulta graficamente,
- fornece limites visíveis no intervalo da consulta,
- fornece uma representação gráfica dos dados e do resultado da consulta,
- fornece feedback imediato do resultado após cada ajuste da consulta,
- e permite que usuários recentes comecem a trabalhar com pouco treinamento.

Uma abordagem comum é manipular parâmetros de consulta usando widgets de interface de usuário padrão, como controles deslizantes, botões de opção e menus suspensos. Para gerar widgets de consulta dinâmica, podemos aplicar a operação `bind` de uma seleção a um ou mais campos de dados que desejamos consultar.

Vamos construir um gráfico de dispersão (_scatter plot_) interativo que usa uma consulta dinâmica para filtrar a exibição. Dado um gráfico de dispersão (_scatter plot_) de classificações de filmes (do Rotten Tomates e do IMDB), podemos adicionar uma seleção sobre o campo `Major_Genre` para permitir a filtragem interativa por gênero de 

Para começar, vamos extrair os gêneros exclusivos (não nulos) dos dados de `movies`:

In [7]:
df = pd.read_json(movies) # load movies data
genres = df['Major_Genre'].unique() # get unique field values
genres = list(filter(lambda d: d is not None, genres)) # filter out None values
genres.sort() # sort alphabetically

Para uso futuro, vamos também definir uma lista de valores exclusivos de `MPAA_Rating`:

In [8]:
mpaa = ['G', 'PG', 'PG-13', 'R', 'NC-17', 'Not Rated']

Agora vamos criar uma seleção `single` vinculada a um menu suspenso.

*Use o menu de consulta dinâmica abaixo para explorar os dados. Como as classificações variam por gênero? Como você revisaria o código para filtrar `MPAA_Rating` (G, PG, PG-13, etc.) ao invés de `Major_Genre`?*

In [9]:
selectGenre = alt.selection_single(
    name='Select', # name the selection 'Select'
    fields=['Major_Genre'], # limit selection to the Major_Genre field
    init={'Major_Genre': genres[0]}, # use first genre entry as initial value
    bind=alt.binding_select(options=genres) # bind to a menu of unique genre values
)

alt.Chart(movies).mark_circle().add_selection(
    selectGenre
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selectGenre, alt.value(0.75), alt.value(0.05))
)

C:\Users\Gustavo\AppData\Local\Temp\ipykernel_28952\2770894913.py:1: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  selectGenre = alt.selection_single(


TypeError: altair.vegalite.v5.schema.core.SelectionParameter() got multiple values for keyword argument 'value'

Nossa construção acima melhora múltiplos aspectos das seleções:

- Damos um nome à seleção (`'Select'`). Este nome não é obrigatório, mas nos permite influenciar o texto do rótulo do menu de consulta dinâmica gerado. (_O que acontece se você remover o nome? Tente!_)
- Restringimos a seleção a um campo de dados específico (`Major_Genre`). Anteriormente, quando usávamos uma seleção `single`, a seleção era mapeada para pontos de dados individuais. Ao limitar a seleção a um campo específico, podemos selecionar _todos_ os pontos de dados cujo valor do campo `Major_Genre` corresponde ao valor único selecionado.
- Inicializamos `init=...` a seleção para um valor inicial.
- `Vinculamos` a seleção a um widget de interface, neste caso um menu suspenso via `binding_select`.
- Como antes, usamos uma codificação condicional para controlar o canal de opacidade.

### Vinculando seleções a múltiplas entradas
---
Uma instância de seleção pode ser vinculada a _múltiplas_ ferramentas (_widgets_) de consulta dinâmica. Vamos modificar o exemplo acima para fornecer filtros para _ambos_ `Major_Genre` e `MPAA_Rating`, usando botões de opção em vez de um menu. Nossa seleção `single` agora é definida sobre um único _par_ de valores de gênero e classificação MPAA

_Procure por conjunções surpreendentes de gênero e classificação. Há algum filme de terror com classificação G ou PG?_


In [ ]:
# single-value selection over [Major_Genre, MPAA_Rating] pairs
# use specific hard-wired values as the initial selected values
selection = alt.selection_single(
    name='Select',
    fields=['Major_Genre', 'MPAA_Rating'],
    init={'Major_Genre': 'Drama', 'MPAA_Rating': 'R'},
    bind={'Major_Genre': alt.binding_select(options=genres), 'MPAA_Rating': alt.binding_radio(options=mpaa)}
)

# scatter plot, modify opacity based on selection
alt.Chart(movies).mark_circle().add_selection(
    selection
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selection, alt.value(0.75), alt.value(0.05))
)

### Usando visualizações como consultas dinâmicas
---

Embora os widgets de interface padrão mostrem os valores de parâmetros de consulta _possíveis_, eles não visualizam a _distribuição_ desses valores. Também podemos desejar usar interações mais ricas, como seleções de vários valores ou intervalos, em vez de widgets de entrada que selecionam um único valor por vez.

Para resolver esses problemas, podemos criar gráficos adicionais para visualizar dados e dar suporte a consultas dinâmicas. Vamos adicionar um histograma da contagem de filmes por ano e usar uma seleção de intervalo para destacar dinamicamente os filmes em períodos de tempo selecionados.

*Interaja com o histograma do ano para explorar filmes de diferentes períodos de tempo. Você viu alguma evidência de [viés de seleção](https://pt.wikipedia.org/wiki/Vi%C3%A9s_de_sele%C3%A7%C3%A3o) ou [sampling bias](https://en.wikipedia.org/wiki/Sampling_bias) ao longo dos anos? (Como o ano e as classificações dos críticos se relacionam?)*

_Os anos variam de 1930 a 2040! Os filmes futuros estão em pré-produção ou há erros de "desvio de um século"? Além disso, dependendo do fuso horário em que você estiver, você poderá ver um pequeno aumento em 1969 ou 1970. Por que isso pode acontecer? (Veja o final do capítulo para uma explicação!)_

In [ ]:
brush = alt.selection_interval(
    encodings=['x'] # limit selection to x-axis (year) values
)

# dynamic query histogram
years = alt.Chart(movies).mark_bar().add_selection(
    brush
).encode(
    alt.X('year(Release_Date):T', title='Films by Release Year'),
    alt.Y('count():Q', title=None)
).properties(
    width=650,
    height=50
)

# scatter plot, modify opacity based on selection
ratings = alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(brush, alt.value(0.75), alt.value(0.05))
).properties(
    width=650,
    height=400
)

alt.vconcat(years, ratings).properties(spacing=5)

O exemplo acima fornece consultas dinâmicas usando uma _linked selection_ entre gráficos:

- Criamos uma seleção `interval` (`brush`) e definimos `encodings=['x']` para limitar a seleção apenas ao eixo x, resultando em um intervalo de seleção unidimensional.
- Registramos `brush` com nosso histograma de filmes por ano via `.add_selection(brush)`.
- Usamos `brush` em uma codificação condicional para ajustar a `opacidade` do gráfico de dispersão (_scatter plot_).

Essa técnica de interação de selecionar elementos em um gráfico e ver destaques vinculados em um ou mais outros gráficos é conhecida como [_brushing &amp; linking_](https://en.wikipedia.org/wiki/Brushing_and_linking).

## Movimentando e Ampliando
---

O gráfico de dispersão (_scatter plot_) da pontuação dos filmes está um pouco confuso em alguns lugares, tornando difícil examinar pontos em regiões com mais densas. Usando as técnicas de interação de _movimentação_ e _ampliação_, podemos inspecionar regiões com maior densidade de pontos de mais perto.

Vamos começar pensando sobre como podemos expressar a movimentação e a ampliação usando as seleções do Altair. O que define a "janela de visualização" do gráfico? _Os domínios das escalas dos eixos!_

Podemos mudar os domínios de escala para modificar a faixa de valores de dados visualizada. Para fazer isso interativamente, podemos vincular uma seleção de intervalo (`interval`) a domínios de escala com o código `bind='scales'`. O resultado é que, ao invés de termos uma região selecionada que pode ser arrastada e ampliada, poderemos agora arrastar e ampliar toda a área do gráfico!

_No gráfico abaixo, clique e arraste para deslocar (traduzir) a visualização, ou role verticalmente para ampliar (escalar). O que você pode descobrir sobre a precisão das avaliações dos filmes providenciadas?_

In [ ]:
alt.Chart(movies).mark_circle().add_selection(
    alt.selection_interval(bind='scales')
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # use min extent to stabilize axis title placement
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
)

_Ampliando, podemos ver que as avaliações tem precisão limitada! As avaliações do Rotten Tomatoes são inteiros, enquanto as do IMDB são separadas por décimos. Como resultado, há pontos sobrepostos mesmo quando ampliamos,  com vários filmes compartilhando a mesma avaliação._

Lendo o código acima, você pode perceber o código `alt.Axis(minExtent=30)` no canal de codificação `y`. O parâmetro `minExtent` garante que uma quantidade mínima de espaço está reservada para marcações no eixo e rótulos. Por que fazer isso? Quando deslocamos e ampliamos, os rótulos do eixo podem mudar e fazer a posição do título do eixo mudar. Ao estabelecer uma extensão mínima, podemos reduzir movimentos que causam distração no gráfico. _Tente mudar o valor do `minExtent`, definindo-o como zero e então diminuindo a ampliação para ver o que acontece quando rótulos de eixo mais longos entram na visualização._

Altair também inclui um atalho para adicionar movimentação e ampliação ao gráfico. Ao invés de criar uma seleção diretamente, você pode chamar `.interactive()` para que o Altair gere  automaticamente um limite para a seleção do intervalo para as escalas do gráfico:

In [ ]:
alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # use min extent to stabilize axis title placement
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
).interactive()

Por padrão, ligação de escala para seleções inclui os canais tanto do eixo `x` quanto do eixo `y`. E se quisermos limitar a movimentação e a ampliação em uma única dimensão? podemos invocar `encodings=['x']` para restringir a seleção somente para o canal `x`:

In [ ]:
alt.Chart(movies).mark_circle().add_selection(
    alt.selection_interval(bind='scales', encodings=['x'])
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # use min extent to stabilize axis title placement
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
)

_Ao ampliar somente ao longo do eixo x, o formato dos dados visualizados pode mudar, potencialmente afetando nossa percepção das relações nos dados. [Escolher uma proporção adequada](http://vis.stanford.edu/papers/arclength-banking) é uma preocupação importante no design de visualizações!_

## Navegação: Visão geral + Detalhes
---

Ao optar por uma visão panorâmica ou um zoom, ajustamos diretamente When panning and zooming, we directly adjust the "janela de visualização" de um gráfico. A estratégia de navegação de _visão geral + detalhes_, pelo contrário usa uma tela de janela de visualização para mostrar _todos_ os dados enquando permite seleções com zoom in e out em uma tela de foco separado.

Abaixo temos dois gráficos de área (_area charts_) mostrando uma década de flutuações de preço para o índice de ações da S&amp;P 500. Inicialmente ambos os gráficos mostram o mesmo intervalo de dados. _Clique e arraste no gráfico de visão geral inferior para atualizar o foco da tela e examinar períodos de tempo específicos._

In [ ]:
brush = alt.selection_interval(encodings=['x']);

base = alt.Chart().mark_area().encode(
    alt.X('date:T', title=None),
    alt.Y('price:Q')
).properties(
    width=700
)

alt.vconcat(
    base.encode(alt.X('date:T', title=None, scale=alt.Scale(domain=brush))),
    base.add_selection(brush).properties(height=60),
    data=sp500
)

Ao contrário do nosso caso anterior de zoom in &amp; out, aqui não queremos vincular uma seleção diretamente às escalas de um único gráfico interativo. Ao invés disso, queremos vincular a seleção a um domínio de escala em _outro_ gráfico. Para fazer isso, atualizamos o canal de codificação `x` para o nosso gráfico de foco, configurando o `domínio` de escala pare referenciar nossa seleção `brush`. Se nenhum intervalo está definido (seleção vazia), o Altair ignora o pincel e usa os dados subjacentes para determinar o domínio. Quando um intervalo de pincel é criado, o Altair o usa em substituição ao `domínio` da escala para o gráfico de interesse.

## Detalhes sob Demanda
---



Uma vez que identificamos pontos de interesse em uma visualização, frequentemente queremos saber mais sobre eles. _Detalhes sob demanda_ refere-se à consulta interativa para obter mais informações sobre valores selecionados. _Tooltips_ são uma forma útil de fornecer detalhes sob demanda. No entanto, os tooltips normalmente mostram informações para um único ponto de dados de cada vez. Como podemos mostrar mais?

O gráfico de dispersão (_scatter plot_) das avaliações de filmes inclui vários outliers potencialmente interessantes, onde as classificações do Rotten Tomatoes e do IMDB discordam. Vamos criar um gráfico que permita selecionar pontos interativamente e mostrar seus rótulos. Para acionar a consulta de filtro em qualquer interação de "hover" ou "click", utilizaremos o operador de composição [Altair](https://altair-viz.github.io/user_guide/interactions.html#composing-multiple-selections) `|` ("or").

_Passando o mouse sobre os pontos no gráfico de dispersão (_scatter plot_) abaixo, você verá um destaque e o título do rótulo. Pressione Shift e clique nos pontos para tornar as anotações persistentes e ver vários rótulos ao mesmo tempo._ Quais filmes são amados pelos críticos do Rotten Tomatoes, mas não pelo público geral no IMDB (ou vice-versa)? Veja se você consegue encontrar possíveis erros, onde dois filmes com o mesmo nome foram acidentalmente combinados!


In [ ]:
hover = alt.selection_single(
    on='mouseover',  # selecionar ao passar o mouse
    nearest=True,    # selecionar o ponto mais próximo do cursor
    empty='none'     # seleção vazia não deve corresponder a nada
)

click = alt.selection_multi(
    empty='none' # seleção vazia não corresponde a nenhum ponto
)

# codificações do gráfico de dispersão compartilhadas por todas as marcas
plot = alt.Chart().mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q'
)

# base compartilhada para novas camadas
base = plot.transform_filter(
    hover | click # filtrar para pontos em qualquer seleção
)

# camadas de pontos do gráfico de dispersão, anotações de halo e rótulos de título
alt.layer(
    plot.add_selection(hover).add_selection(click),
    base.mark_point(size=100, stroke='firebrick', strokeWidth=1),
    base.mark_text(dx=4, dy=-8, align='right', stroke='white', strokeWidth=2).encode(text='Title:N'),
    base.mark_text(dx=4, dy=-8, align='right').encode(text='Title:N'),
    data=movies
).properties(
    width=600,
    height=450
)


O exemplo acima adiciona três novas camadas ao gráfico de dispersão: uma anotação circular, texto branco para fornecer um fundo legível e texto preto mostrando o título de um filme. Além disso, este exemplo utiliza duas seleções em conjunto:

1. Uma seleção única (`hover`) que inclui `nearest=True` para selecionar automaticamente o ponto de dados mais próximo à medida que o mouse se move.
2. Uma seleção múltipla (`click`) para criar seleções persistentes por meio do shift-click.

Ambas as seleções incluem o conjunto `empty='none'` para indicar que nenhum ponto deve ser incluído se a seleção estiver vazia. Essas seleções são então combinadas em um único predicado de filtro — o lógico _ou_ de `hover` e `click` — para incluir pontos que pertencem a _qualquer uma_ das seleções. Usamos esse predicado para filtrar as novas camadas e exibir anotações e rótulos apenas para os pontos selecionados.


Usando seleções e camadas, podemos realizar uma série de designs diferentes para detalhes sob demanda! Por exemplo, aqui está uma série temporal com escala logarítmica dos preços das ações de tecnologia, anotada com uma linha de guia e rótulos para a data mais próxima do cursor do mouse:


In [ ]:
# selecione um ponto para o qual fornecer detalhes sob demanda
label = alt.selection_single(
    encodings=['x'], # limitar seleção ao valor no eixo x
    on='mouseover',  # selecionar em eventos de passar o mouse
    nearest=True,    # selecionar o ponto de dados mais próximo ao cursor
    empty='none'     # seleção vazia não inclui pontos de dados
)

# defina nosso gráfico base de linha dos preços das ações
base = alt.Chart().mark_line().encode(
    alt.X('date:T'),
    alt.Y('price:Q', scale=alt.Scale(type='log')),
    alt.Color('symbol:N')
)

alt.layer(
    base, # gráfico base de linha

    # adicione uma marca de linha para servir como linha guia
    alt.Chart().mark_rule(color='#aaa').encode(
        x='date:T'
    ).transform_filter(label),

    # adicione marcas de círculo para os pontos de tempo selecionados, esconda pontos não selecionados
    base.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_selection(label),

    # adicione texto com borda branca para fornecer um fundo legível para os rótulos
    base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
        text='price:Q'
    ).transform_filter(label),

    # adicione rótulos de texto para os preços das ações
    base.mark_text(align='left', dx=5, dy=-5).encode(
        text='price:Q'
    ).transform_filter(label),

    data=stocks
).properties(
    width=700,
    height=400
)


_Pondo na prática o que aprendemos até agora: você pode modificar o gráfico de dispersão de filmes acima (aquele com a consulta dinâmica ao longo dos anos) para incluir uma marca `rule` que mostre a média da avaliação do IMDB (ou Rotten Tomatoes) para os dados contidos na seleção do intervalo de `ano`?_


## Revisitando o "Brushing & Linking"
---

Mais cedo nesse capítulo vimos um exemplo de "*brushing and linking*" (algo como *podar e conectar*, em português): usando um histograma de query dinâmico para realçar pontos num scatter plot das notas de filmes. Aqui, vamos visitar alguns exemplos adicionais envolvendo seleções linkadas.

Retornando a base de dados do `cars`, podemos usar o operador `repeat` para criar uma [matriz de gráfico de dispersão (ou SPLOM)](https://pt.wikipedia.org/wiki/Gr%C3%A1fico_de_dispers%C3%A3o) que mostra as associações entre a aceleração, potência e a quilometragem. Podemos definir uma seleção do intervalo (`interval`) e incluir isso dentro das especificações repetidas do gráfico de dispersão para conseguirmos ter seleções conectadas em todas as visualizações.

*Clique e arraste em qualquer uma das visualizações individuais para fazer o brushing & linking!*

In [ ]:
brush = alt.selection_interval(
    resolve='global' # resolve all selections to a single global instance
)

alt.Chart(cars).mark_circle().add_selection(
    brush
).encode(
    alt.X(alt.repeat('column'), type='quantitative'),
    alt.Y(alt.repeat('row'), type='quantitative'),
    color=alt.condition(brush, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(brush, alt.value(0.8), alt.value(0.1))
).properties(
    width=140,
    height=140
).repeat(
    column=['Acceleration', 'Horsepower', 'Miles_per_Gallon'],
    row=['Miles_per_Gallon', 'Horsepower', 'Acceleration']
)

Note que no código acima usamos o `resolve='global'` no intervalo (`interval`) de seleção. A configuração padrão do `'global'` indica que entre todas as visualizações apenas uma seleção pode estar ativa ao mesmo tempo. Entretanto, em alguns casos queremos ter múltiplas seleções em múltiplas visualizações e juntar os resultados. Se nós usarmos `resolve='union'`, a seleção principal vai conter a *união* de todas as seleções: se um ponto estiver dentro da união de seleções ele será selecionado. Alternativamente, se usarmos o `resolve='intersect'`, a seleção principal vai consistir na intersecção de todas as seleções individuais: apenas os pontos nessa intersecção serão contabilizados e selecionados.

*Tente mudar o parâmetro `resolve` para `'union'` ou `'intersect'` para ver como ele muda a lógica de seleção resultante.*

## Resumo
---

Para mais detalhes sobre as opções de interação disponíveis em Altair, por facor consulte a [Documentação de seleção interativa para Altair](https://altair-viz.github.io/user_guide/interactions.html). Para detalhes sobre como customizar manipuladores de eventes, por exemplo para compor técnicas de múltiplas intereções ou aceitar inputs de toque para dispositivos touch-screen, veja a [documentação de seleção Vega-Lite](https://vega.github.io/vega-lite/docs/selection.html).

Interessado em aprender mais?
- A abstração de _seleção_ foi introduzida no artigo [Vega-Lite: A Grammar of Interactive Graphics](http://idl.cs.washington.edu/papers/vega-lite/), por Satyanarayan, Moritz, Wongsuphasawat, &amp; Heer.
- O sistema PRIM-9 (para projeções, rotações, isolamento e masking de até 9 dimensõess) é uma das primeiras ferramentas de visualização interativa, construída no início dos anos 70 por Fisherkeller, Tukey, &amp; Friedman. [Um vídeo retrô de demonstração sobreviveu!](http://stat-graphics.org/movies/prim9.html)
- O conceito de pincelar e &amp; linkar foi cristalizado por Becker, Cleveland, &amp; Wilks em seu artigo de 1987 [Dynamic Graphics for Data Analysis](https://scholar.google.com/scholar?cluster=14817303117298653693).
- Para um sumário compreensivo de técnicas de interação para visualizações, veja [Interactive Dynamics for Visual Analysis](https://queue.acm.org/detail.cfm?id=2146416) por Heer &amp; Shneiderman.
- Finalmente, para um tratado sobre o que torna uma interalçao efetiva leia o clássico [Direct Manipulation Interfaces](https://scholar.google.com/scholar?cluster=15702972136892195211) de Hutchins, Hollan, &amp; Norman.